In [ ]:
"""society_ranges = {
    "Society_1": {"qty": (150, 200), "fat": (3.9, 4.1), "snf": (7.9, 8.2)},
    "Society_2": {"qty": (100, 130), "fat": (3.6, 3.9), "snf": (7.7, 7.9)},
    "Society_3": {"qty": (80, 110), "fat": (3.5, 3.8), "snf": (7.6, 7.8)},
    "Society_4": {"qty": (120, 150), "fat": (3.7, 4.0), "snf": (7.8, 8.0)},
    "Society_5": {"qty": (90, 140), "fat": (3.5, 3.9), "snf": (7.7, 7.9)}
}"""

In [8]:
def creation_train_model(society_ranges,days):
    import pandas as pd
    import numpy as np
    society=list(society_ranges.keys())
    df=pd.DataFrame({'Date':pd.date_range(start='2025-08-01',periods=days,freq='D')})
    #qty=np.random.randint(r[0],r[1],60)
    import random
    random.seed(42)

    for name,r in society_ranges.items():
        qty=np.random.randint(r['qty'][0],r['qty'][1],days)
        fat=np.round(np.random.uniform(r['fat'][0],r['fat'][1],days),1)
        snf=np.round(np.random.uniform(r['snf'][0],r['snf'][1],days),1)
        ts=np.round((fat+snf),1)
        weighted_fat=np.round(qty*fat/100,3)
        weighted_snf=np.round(qty*snf/100,3)
        weighted_average=np.round((qty*(fat+snf)/100),3)
        df[f"{name}_QTY"]=qty
        df[f"{name}_Fat"]=fat
        df[f"{name}_Snf"]=snf
        df[f"{name}_TS"]=ts
        df[f"{name}_Weighted_Fat"]=weighted_fat
        df[f"{name}_Weighted_Snf"]= weighted_snf
        df[f"{name}_Weighted_Average"]=weighted_average
    df['Overall_Weighted_Average']=df[[f'{s}_Weighted_Average' for s in society]].sum(axis=1)
    df['Overall_Weighted_Fat']=df[[f"{s}_Weighted_Fat" for s in society]].sum(axis=1)
    df['Overall_Weighted_Snf']=df[[f"{s}_Weighted_Snf" for s in society]].sum(axis=1)
    df['Total_Qty']=df[[f"{s}_QTY" for s in society]].sum(axis=1)
    df['Overall_Fat'] = np.round((df['Overall_Weighted_Fat'] * 100) / df['Total_Qty'], 1)
    df['Overall_SNF'] = np.round((df['Overall_Weighted_Snf'] * 100) / df['Total_Qty'], 1)
    df['Overall_TS']=np.round((df['Overall_Fat']+df['Overall_SNF']),1)
    
    #print(df.head(10))

    qty_col=[f"{s}_QTY" for s in society]

    x_fat=df[qty_col+['Overall_Weighted_Fat','Overall_Fat']]
    y_fat=df[[f"{s}_Weighted_Fat" for s in society]]

    x_Snf=df[qty_col+['Overall_Weighted_Snf','Overall_SNF']]
    y_Snf=df[[f"{s}_Weighted_Snf" for s in society]]

    x_WA=df[qty_col+['Overall_Weighted_Average','Overall_TS']]
    y_WA=df[[f"{s}_Weighted_Average" for s in society]]

    def split(x,y):
        from sklearn.model_selection import train_test_split
        return train_test_split(x,y,test_size=1/3,random_state=0)

    x_train_f,x_test_f,y_train_f,y_test_f=split(x_fat,y_fat)
    x_train_snf,x_test_snf,y_train_snf,y_test_snf=split(x_Snf,y_Snf)
    x_train_wa,x_test_wa,y_train_wa,y_test_wa=split(x_WA,y_WA)

    def scale(x_train,x_test,y_train,y_test):
        from sklearn.preprocessing import StandardScaler
        xscaler=StandardScaler()
        yscaler=StandardScaler()
        x_train_sc=xscaler.fit_transform(x_train)
        x_test_sc=xscaler.transform(x_test)
        y_train_sc=yscaler.fit_transform(y_train)
        y_test_sc=yscaler.transform(y_test)
        return x_train_sc,x_test_sc,y_train_sc,y_test_sc,xscaler,yscaler

    x_train_f_sc,x_test_f_sc,y_train_f_sc,y_test_f_sc,xscaler_fat,yscaler_fat=scale(x_train_f,x_test_f,y_train_f,y_test_f)
    x_train_snf_sc,x_test_snf_sc,y_train_snf_sc,y_test_snf_sc,xscaler_snf,yscaler_snf=scale(x_train_snf,x_test_snf,y_train_snf,y_test_snf)
    x_train_wa_sc,x_test_wa_sc,y_train_wa_sc,y_test_wa_sc,xscaler_WA,yscaler_Wa=scale(x_train_wa,x_test_wa,y_train_wa,y_test_wa)

    import joblib
    fat_in_sc="Synthetic_Fat_input_Scaler.joblib"
    fat_out_sc="Synthetic_Fat_output_Scaler.joblib"
    snf_in_sc="Synthetic_Snf_input_Scaler.joblib"
    snf_out_sc="Synthetic_Snf_output_Scaler.joblib"
    wa_in_sc="Synthetic_WA_input_Scaler.joblib"
    wa_out_sc="Synthetic_WA_output_Scaler.joblib"
    joblib.dump(xscaler_fat,fat_in_sc)
    joblib.dump(yscaler_fat,fat_out_sc)
    joblib.dump(xscaler_snf,snf_in_sc)
    joblib.dump(yscaler_snf,snf_out_sc)
    joblib.dump(xscaler_WA,wa_in_sc)
    joblib.dump(xscaler_WA,wa_out_sc)

    def model_evaluation(x_train_sc,x_test_sc,y_train_sc,y_test_sc,name):
        from sklearn.linear_model import RidgeCV
        from sklearn.multioutput import MultiOutputRegressor as MOR
        from sklearn.metrics import r2_score, mean_absolute_error
        alpha=[0.1,1.0,10.0,100.0]
        model=MOR(RidgeCV(alphas=alpha,cv=5,scoring='neg_mean_squared_error'))
        model.fit(x_train_sc,y_train_sc)
        y_pred=model.predict(x_test_sc)
        r_score=r2_score(y_test_sc,y_pred)
        mae=mean_absolute_error(y_test_sc,y_pred)
        print(f"{name} Model → R²: {r_score:.4f}, MAE: {mae:.4f}")
        return model,r_score,mae

    fat_model,r_score_f,mae_f=model_evaluation(x_train_f_sc,x_test_f_sc,y_train_f_sc,y_test_f_sc,"Fat")
    snf_model,r_score_snf,mae_snf=model_evaluation(x_train_snf_sc,x_test_snf_sc,y_train_snf_sc,y_test_snf_sc,"SNF")
    Weighted_average_model,r_score_wa,mae_wa=model_evaluation(x_train_wa_sc,x_test_wa_sc,y_train_wa_sc,y_test_wa_sc,"Weighted Average")

    fat_pred="Fat_Prediction_synthetic.joblib"
    snf_pred="Snf_Prediction_synthetic.joblib"
    wa_pred="Weighted_Average_Prediction_synthetic.joblib"
    joblib.dump(fat_model,fat_pred)
    joblib.dump(snf_model,snf_pred)
    joblib.dump(Weighted_average_model,wa_pred)
    

    
    return {"data": df,"metrics": {"Fat": {"R2": r_score_f, "MAE": mae_f},"SNF": {"R2": r_score_snf, "MAE": mae_snf},"Weighted_Avg": {"R2": r_score_wa, "MAE": mae_wa}}}
    
   

   

In [9]:
society_ranges = {
    "Society_1": {"qty": (150, 200), "fat": (3.9, 4.1), "snf": (7.9, 8.2)},
    "Society_2": {"qty": (100, 130), "fat": (3.6, 3.9), "snf": (7.7, 7.9)},
    "Society_3": {"qty": (80, 110), "fat": (3.5, 3.8), "snf": (7.6, 7.8)},
    "Society_4": {"qty": (120, 150), "fat": (3.7, 4.0), "snf": (7.8, 8.0)},
    "Society_5": {"qty": (90, 140), "fat": (3.5, 3.9), "snf": (7.7, 7.9)}}

In [10]:
data,metrics=creation_train_model(society_ranges,60)

Fat Model → R²: 0.9071, MAE: 0.2324
SNF Model → R²: 0.9877, MAE: 0.0794
Weighted Average Model → R²: 0.9809, MAE: 0.1031
